# mesos的master  
我学习的是**mesos-0.13.0**版本。  
从 **src/master/master.cpp、master.hpp** 开始看。记录的是我自己的理解，有些地方可能理解的不到位，写的有问题，仅供参考。  

## namespace master
master 命名空间。
```C++
class Allocator;
class SlaveObserver;
class WhitelistWatcher;
class Master;

struct Framework;
struct Slave;
```
---

# 启动Master服务
启动Master服务由Master的main函数进行。
## 1. 初始化 Master对象。
Master本身作为一个 libprocess 的 process对象 需要进行初始化.
### class Master  
这里的 Master类 作为 libprocess 中的一个 **process** 记成了 ProtobufProcess类 ，继而继承了 Process类。Master 的对象将会作为一个长期的 socket server 服务进程，监听其他的 process 的消息并根据已注册的各类消息的响应函数作出反馈。
>ProtobufProcess类 的作用是使 Process类 支持 Message 的传递  
![mesos-master.png](./pictures/mesos-master.png)

```C++
class Master : public ProtobufProcess<Master>{
//...
}
```  
### **process::initialize()**  
指的是 process 命名空间下的 initialize() 函数，在 process.cpp 中定义。**为libprocess模型中的每一个 process对象 （例如：master-process/Master，slave-process/Slave，framework-process/MesosSchedulerDriver，executor-process）做初始化操作**。  一个 process对象 只进行一次初始化操作。如果当前 process 已初始化，则不会继续初始化。  
做的事情如下（简单的提了关键的几个步骤，具体详见$MESOS_ROOT/3rdpardy/libprocess/src/process.cpp中的代码）： 

> 1.创建 ProcessManager 和 SocketManager
>```C++
>  //step1
>  process_manager = new ProcessManager(mytest);  
>  socket_manager = new SocketManager();
>  ```
> >  **class ProcessManager：**
> >   ```C++
> >    // Gates for waiting threads (protected by synchronizable(processes)).
> >    map<ProcessBase*, Gate*> gates;
> >     // Queue of runnable processes (implemented using list).
> >     list<ProcessBase*> runq;
> > ``` 

> 2. 启动 processing threads
> 有几个 cpu 个数，启动几个 processing threads
> ```C++
>  //step2
> // Setup processing threads.
>   long cpus = std::max(4L, sysconf(_SC_NPROCESSORS_ONLN));
>   for (int i = 0; i < cpus; i++) {
>       pthread_t thread; // For now, not saving handles on our threads.
>   }
> ```
> 3. 获取 libprocess 的 IP、Port 号等信息，创建socket服务
> ```C++
> //step3
>  value = getenv("LIBPROCESS_IP");
>  value = getenv("LIBPROCESS_PORT");
> ```
> 4. **创建 socket server**
> 这里建立 socket 通信的基本模型，将 socket 与之前获取的 libprocess 的ip地址，port号绑定。并开始监听当前端口。
> 
> 5. 使用 libprocess::spawn() 函数启动当前 process 的全局的 garbage collector-process ， logging-process ， profiler-process。 
> 这里的 libprocess::spawn() 中又调用了**process_manager** 的 spawn() 函数，将启动的 process 加入 process 的 **run queue** 中。
> `当 process 一旦被放入了 runq 中去，gate->open() 方法唤醒处理线程，则 socket server 会响应这个 process 的 initialize() 方法。  
> >```C++
> >  //process.cpp
> >  // Add process to the run queue (so 'initialize' will get invoked).
> >  enqueue(process);
> >```
> >```C++
> > void ProcessManager::enqueue(ProcessBase* process)
> > {
> >      //...
> >      // Wake up the processing thread if necessary.
> >      gate->open();
> > }
> >```
> **我的理解：** 这里的每一个 process 都是一类消息的处理器，都被 libprocess 存放到一个队列中（这个**队列**是step1中提到的，ProcessManager类中的属性：runq）。当前服务接收到消息后会依次遍历队列中的处理器，是否能处理当前消息，如果可以则处理，不可以则访问下一个处理器。
> >[来自网友](https://blog.csdn.net/qianggezhishen/article/details/52049949)
> >LibProcess 采用了基于事件驱动的编程模型，每一个服务（进程）内部实际上运行了一个 socket server，而不同服务之间通过消息（事件）进行通信。在一个服务内部，注册了很多消息以及每个消息对应的处理器，一旦它收到某种类型的消息，则会调用相应的处理器进行处理，在处理过程中，可能会产生另外一种消息发送给另一个服务。整个过程如下图所示。  
> >![libprocess.png](./pictures/libprocess.png)
> 6. 初始化其他服务（http、mime等）

## 2. 启动 master 的服务  
将 master 作为一个 process 启动（process::spawn）。将 master 放入 runq 队列之后，libprocess 就会调用 master 的 initialize() 函数，注册消息响应函数。与 process::initialize() 中的步骤5一样。